<a href="https://colab.research.google.com/github/DLPY/Regression-Session-2/blob/master/SydneyHousePrices_MultipleRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<centre> **House Price Prediction based on Postal Code, Number of Bedrooms and Bathrooms, Car Parking  and Property Type**</centre>

Detail on Data : https://www.kaggle.com/mihirhalai/sydney-house-prices

## 1. Download source data from Github

In [ ]:
!wget https://raw.githubusercontent.com/DLPY/Regression-Session-2/master/Data/SydneyHousePrices.csv

## 2. Import neccssary packages for performing EDA and Multiple regression

In [ ]:
# Imports
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
%matplotlib inline
pd.set_option('display.max_colwidth', None)

## 3. Read data into pandas dataframe to perform data analysis, cleaning and transformation

In [ ]:
df = pd.read_csv('SydneyHousePrices.csv')

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
round(df.describe(),2)

**From the above max - It shows the dataset has outliers and it needs to be removed.**

## 4. Choosing predictors and target variables for performing Multiple Regression

**Target and Source variables**

*   SellingPrice - Target Variable
*   Bed, Bath, Car, propType, postalCode  - Predictor Variables



In [ ]:
df_new = df[['postalCode', 'bed', 'bath', 'car', 'propType' , 'sellPrice']]

In [ ]:
df_new.propType.unique()

In [ ]:
df_new.head(5)

Encoding the categorical variables - Change the text into numbers

In [ ]:
df_new['propType'] = df_new['propType'].astype('category').cat.codes

In [ ]:
df_new.propType.unique()

In [ ]:
df_new.head(5)

In [ ]:
df_new.dtypes

In [ ]:
df_new.count()

In [ ]:
df_new.replace(np.nan, 0)

## 5. Remove outliers in the data

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [ ]:
df_no_outlier_bed=remove_outlier(df_new, 'bed').reset_index(drop=True)
df_no_outlier_bath=remove_outlier(df_no_outlier_bed, 'bath').reset_index(drop=True)
df_no_outlier_car=remove_outlier(df_no_outlier_bath, 'car').reset_index(drop=True)
df_no_outlier=remove_outlier(df_no_outlier_car, 'sellPrice').reset_index(drop=True)

In [ ]:
df_no_outlier

In [ ]:
corr = df_no_outlier.corr()

In [ ]:
corr

## 6. Correlation heatmap with the mask and correct aspect ratio

In [ ]:
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(10, 250, as_cmap=True)
f, ax = plt.subplots(figsize=(11, 9))
sns.heatmap(corr, mask=mask, cmap=cmap,
            square=True, linewidths=.2, cbar_kws={'shrink': .5}, ax=ax, annot=True)

## 7. Split Target and Predictor Variables to different dataframes

In [ ]:
X = df_no_outlier.iloc[:,:-1]
Y = df_no_outlier.iloc[:,5]

In [ ]:
X.head(5)

In [ ]:
Y.head(5)

Convert dataframes to values to feed into Model

In [ ]:
X = X.values
Y= Y.values

In [ ]:
print('Number of records and predictor variables:', X.shape)
print('Number of records and target variable:', Y.shape)

## 8. Split dataset into the training and test using train_set_split: 90% - train and 10% - test

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.1, random_state=23)

In [ ]:
print ('Training Data:',X_train.shape, Y_train.shape)
print ('Testing Data:',X_test.shape, Y_test.shape)

## 9. Train, Test and Predict using regression model

Create an object called regressor in the LinearRegression class

In [ ]:
regressor = LinearRegression()

Fit the linear regression model to the training set. We use the fit method the arguments of the fit method will be training sets

In [ ]:
regressor.fit(X_train,Y_train)

Regression Coefficients

In [ ]:
print('Coefficients: ', regressor.coef_)

In [ ]:
columns=['postalCode', 'bed', 'bath', 'car', 'propType' ]
Coefficient = pd.DataFrame(regressor.coef_).T
Coefficient.columns = columns
Coefficient

Predicting the Test set results

In [ ]:
Y_pred= regressor.predict(X_test)

## 10. Evaluation metrics - How to Calculate R-Square and RMSE

In [ ]:
coefficient_of_dermination = r2_score(Y_test,Y_pred)
print('R-squared:',coefficient_of_dermination)

rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
print('Root Mean Squared Error: {}'.format(rmse))

In [ ]:
# Displaying Results and Difference in Table 
res = pd.DataFrame(Y_pred, Y_test)
res = res.reset_index()
res.columns = ['Price', 'Prediction']
res['Prediction'] = round(res['Prediction'],0)
res['Difference'] = res['Price'] - res['Prediction']
res.head(5)

## 11. Regression on Full data using OLS model

In [ ]:
Regression = sm.OLS(endog = Y, exog = X).fit()

In [ ]:
print(Regression.summary())

###To read more on skewness, kurtosis, autocorrelation and multicollinearity.

---
https://www.sciencedirect.com/topics/neuroscience/kurtosis 

---
https://www.investopedia.com/terms/d/durbin-watson-statistic.asp 

---
https://www.analyticsvidhya.com/blog/2020/03/what-is-multicollinearity